# Kỹ Thuật Imputation Trên Bộ Dữ Liệu Titanic
Notebook này trình bày toàn bộ các bước xử lý giá trị thiếu (missing values).

##  1. Import thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

##  2. Tải bộ dữ liệu Titanic

In [ ]:
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)
df.head(20)

##  3. Kiểm tra giá trị thiếu

In [ ]:
df.isna().sum()

### Biểu đồ giá trị thiếu theo cột

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=df.isna().sum().index, y=df.isna().sum().values)
plt.xticks(rotation=45)
plt.title('Số lượng giá trị thiếu trên từng cột')
plt.show()

## 4. Imputation đơn biến (Univariate)

In [ ]:
median_imp = SimpleImputer(strategy='median')
df['Age_median'] = median_imp.fit_transform(df[['Age']])

mode_imp = SimpleImputer(strategy='most_frequent')
df['Embarked_mode'] = mode_imp.fit_transform(df[['Embarked']]).ravel()

const_imp = SimpleImputer(strategy='constant', fill_value='Unknown')
df['Cabin_constant'] = const_imp.fit_transform(df[['Cabin']]).ravel()

print("\n=== Age===")
display(df[['Age', 'Age_median']][df['Age'].isna()].head(10))

print("\n=== Embarked ===")
display(df[['Embarked', 'Embarked_mode']][df['Embarked'].isna()].head(10))

print("\n=== Cabin ===")
display(df[['Cabin', 'Cabin_constant']][df['Cabin'].isna()].head(10))

df_showcase = df[df[['Age', 'Embarked', 'Cabin']].isna().any(axis=1)][[
    'Age','Age_median',
    'Embarked','Embarked_mode',
    'Cabin','Cabin_constant'
]]

print("\n === Bảng tổng hợp ===")
display(df_showcase.head(20))

### So sánh phân phối Age trước và sau Mean Imputation

In [ ]:
plt.figure(figsize=(10,5))
sns.kdeplot(df['Age'], label='Age gốc', fill=True)
sns.kdeplot(df['Age_median'], label='Age sau Median Imputation', fill=True)
plt.legend()
plt.title('So sánh phân phối Age trước và sau Imputation')
plt.show()

### Chuẩn hoá các dữ liệu cần thiết

In [ ]:
df['Sex_encoded'] = df['Sex'].map({'male': 0, 'female': 1})

df_filtered = df[['PassengerId','Survived','Age','Sex','Sex_encoded','Name']]

df_filtered.head()


## 5. Imputation đa biến — Regression

In [ ]:

df_reg = df.copy()

# Train data
train_df = df_reg[df_reg['Age'].notna()]

# Test data
test_df  = df_reg[df_reg['Age'].isna()]

# Dự đoán
X_train = train_df[['Fare','Pclass','Sex_encoded','SibSp','Parch']]
y_train = train_df['Age']

X_test  = test_df[['Fare','Pclass','Sex_encoded','SibSp','Parch']]

# Train
reg = LinearRegression()
reg.fit(X_train, y_train)

# Predict Age for missing rows
df_reg.loc[df_reg['Age'].isna(), 'Age_reg'] = reg.predict(X_test)

print("\n=== Age (Regression Imputation) ===")
df_reg_showcase = df_reg[df['Age'].isna()][[
    'PassengerId','Survived','Age','Age_reg','Sex','Name'
]]

display(df_reg_showcase.head(15))


## 6. Imputation đa biến — KNN

In [ ]:
selected_cols = ['Age','Fare','Pclass','Sex_encoded','SibSp','Parch']
df_selected = df[selected_cols]

knn_full = KNNImputer(n_neighbors=5)
df_knn_full = knn_full.fit_transform(df_selected)

df_knn_df = pd.DataFrame(df_knn_full, columns=[col + "_knn" for col in df_selected.columns])

df_knn_compare = pd.concat([df, df_knn_df], axis=1)

print("\n=== Age ===")
df_knn_showcase = df_knn_compare[df[['Age']].isna().any(axis=1)][['PassengerId','Survived','Age','Age_knn','Sex_encoded','Name',]]

display(df_knn_showcase.head(15))

##  7. Imputation đa biến — MICE

In [ ]:
mice = IterativeImputer(max_iter=20, random_state=0)
df_mice_full = mice.fit_transform(df_selected)

df_mice_df = pd.DataFrame(df_mice_full, columns=[col + "_mice" for col in df_selected.columns])

df_mice_compare = pd.concat([df, df_mice_df], axis=1)

print("\n=== Age ===")
df_mice_showcase = df_mice_compare[df[['Age']].isna().any(axis=1)][['PassengerId','Survived','Age','Age_mice','Sex_encoded','Name']]

display(df_mice_showcase.head(15))


##  8. Gộp toàn bộ kết quả

In [ ]:
final_df = pd.concat([df,df_reg, df_knn_df, df_mice_df], axis=1)
final_df.head()

##  9. Kết luận
- Mean/Median/Mode phù hợp các trường hợp đơn giản.
- KNN giúp tận dụng cấu trúc dữ liệu.
- MICE tái tạo phân phối dữ liệu tốt nhất nhưng chậm hơn.
- Titanic Dataset rất phù hợp minh hoạ các kỹ thuật imputation.

##  10. So sánh trực quan các phương pháp Imputation

### So sánh phân phối Age: Mean vs KNN vs MICE

In [ ]:
plt.figure(figsize=(10,6))
sns.kdeplot(df["Age"], label="Age gốc", fill=True)
sns.kdeplot(df["Age_median"], label="Median Imputation", fill=True)
sns.kdeplot(df_reg["Age_reg"], label="Regression Imputation", fill=True)
sns.kdeplot(df_knn_df["Age_knn"], label="KNN Imputation", fill=True)
sns.kdeplot(df_mice_df["Age_mice"], label="MICE Imputation", fill=True)
plt.title("So sánh phân phối Age của các phương pháp Imputation")
plt.legend()
plt.show()